In [216]:
import pandas as pd
import numpy as np

In [217]:
# Read in the dataset as a pandas DataFrame
dataset = pd.read_csv('bank_marketing.csv')
dataset.head(5)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [218]:
# Split the data into three DataFrames
# The first will contain information about the client
client = dataset[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]
client.head(5)

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [219]:
# The second DataFrame will contain campaign data
campaign = dataset[["client_id", "campaign", "month", "day", "duration", "pdays", "previous", "poutcome", "y"]]
campaign.head(5)

,client_id,campaign,month,day,duration,pdays,previous,poutcome,y
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no


In [220]:
# The third DataFrame will store information about economics at the time of the campaign
economics = dataset[["client_id", "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]]
economics.head(5)

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [221]:
# Renaming columns
new_colnames_client = {"client_id": "id"}
client = client.rename(columns = new_colnames_client)

new_colnames_campaign = {"duration": "contact_duration", 
                         "previous": "previous_campaign_contacts", 
                         "y": "campaign_outcome", 
                         "poutcome": "previous_outcome", 
                         "campaign": "number_contacts"}
campaign = campaign.rename(columns = new_colnames_campaign)

new_colnames_economics = {"euribor3m": "euribor_three_months", 
                          "nr_employed": "number_employed"}
economics = economics.rename(columns = new_colnames_economics)

In [222]:
# Replacing values in the client DataFrame
client["education"] = client["education"].str.replace(".", "_")
client = client.replace("unknown", np.NaN)
client["job"] = client["job"].str.replace(".", "")
print(client.dtypes)
client.head(5)

id                 int64
age                int64
job               object
marital           object
education         object
credit_default    object
housing           object
loan              object
dtype: object


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [223]:
# Replacing values in the campaign DataFrame
campaign = campaign.replace("success", 1)
campaign = campaign.replace("failure", 0)
campaign = campaign.replace("nonexistent", np.NaN)
campaign.head(5)

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,NaN,no
1,1,1,may,19,149,999,0,NaN,no
2,2,1,may,23,226,999,0,NaN,no
3,3,1,may,27,151,999,0,NaN,no
4,4,1,may,3,307,999,0,NaN,no


In [224]:
# Creating new columns in the campaign DataFrame
campaign["campaign_id"] = "1"
campaign["month"] = campaign["month"].str.capitalize()
campaign["year"] = "2022"
campaign["day"] = campaign["day"].astype("string")
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = "%Y-%b-%d")
campaign["last_contact_date"] = campaign["last_contact_date"].dt.strftime("%Y-%b-%d")
campaign = campaign.drop(["month", "day", "year"], axis = 1)
print(campaign.dtypes)
campaign.head(5)

client_id                       int64
number_contacts                 int64
contact_duration                int64
pdays                           int64
previous_campaign_contacts      int64
previous_outcome              float64
campaign_outcome               object
campaign_id                    object
last_contact_date              object
dtype: object


,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,no,1,2022-May-13
1,1,1,149,999,0,NaN,no,1,2022-May-19
2,2,1,226,999,0,NaN,no,1,2022-May-23
3,3,1,151,999,0,NaN,no,1,2022-May-27
4,4,1,307,999,0,NaN,no,1,2022-May-03


In [225]:
# Saving the data into three separate csv files
client.to_csv("client.csv", index = False)
campaign.to_csv("campaign.csv", index = False)
economics.to_csv("economics.csv", index = False)

In [226]:
# Creating the client table
client_table = """CREATE TABLE client 
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER
"""

In [227]:
# Creating the campaign table
campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [228]:
# Creating the economics table
economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT,
);
\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER
"""